In [2]:
from pprint import pprint
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import csv

# JSON을 CSV로 변경 함수

In [3]:
def json_to_csv(jsonFileName, wantCsvFileName):  # 루트 경로가 아니면 경로까지 추가 필요
    # json 파일 읽어오기
    with open(jsonFileName) as f:
        json_data = json.load(f)
    # json파일을 csv로 변경하여 저장
    csv_data = open(wantCsvFileName, 'w', encoding='utf-8')
    csvwriter = csv.writer(csv_data)
    count = 0
    for kk in json_data:
        if count == 0:
            header = kk.keys()
            csvwriter.writerow(header)
            count += 1
        csvwriter.writerow(kk.values())
    csv_data.close()

json_to_csv('data.txt', 'test.csv')

# csv형식의 파일을 로드하고 원하는 특징을 이용해 수정

In [6]:
df = pd.read_csv("movie.csv")
features = ['genreAlt']  # 원하는 속성 입력
def combine_features(row):
    result = ''
    for c in features:
        result += ' '.join(row['genreAlt'].split(','))
    return result
# NaN을 빈 string로 채움
for feature in features:
    df[features] = df[features].fillna('')  # NaN을 ''로 변경하여 채워줌
df['combined_features'] = df.apply(combine_features, axis=1) # combine feature df에 추가

# 영화 정보를 넣었을 경우 가장 비슷한 형식의 영화 정보 출력 함수

In [64]:
# 영화 인덱스를 이용한 영화제목 출력함수
def get_title_from_index(where, index):
    return where['movieNm'][index]
# 영화 제목을 이용한 인덱스 출력 함수
def get_index_from_title(where, movieNm):
    return where.loc[df['movieNm'] == movieNm].index[0]

def movie_recommend(MovieNames, others, nums): # 두개의 파라미터 다 리스트 형식으로 입력
    all_combined_features = []
    for name in MovieNames:
        all_combined_features += df['combined_features'][get_index_from_title(df, name)].split()
    all_combined_features += others
    tmp_df = df.append({'combined_features' : ' '.join(set(all_combined_features))}, ignore_index=True)
    
    # countVector 유사도 검사
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(tmp_df['combined_features'])
    cosine_sim = cosine_similarity(count_matrix)
    index = df.shape[0]
    similar_movies = list(enumerate(cosine_sim[index]))
    sorted_similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    
    result = []
    
    
    for e in sorted_similar_movies:
        if e[1] < 0.3 or len(result) == nums:
            break
        title = get_title_from_index(tmp_df, e[0])
#         print(title)
#         print(type(title))
#         print(title in MovieNames)
#         print(title is not str)
        if (title in MovieNames) or str(title) == 'nan':
            continue
        
        result.append(title)
    return result

In [65]:
a = movie_recommend(['허슬러'], [], 10)
print(a)

['싱', '너의 여자친구']
